In [206]:
# You may need to install htmllib5,lxml, and BeautifulSoup4. In your terminal/command prompt run:

# conda install lxml
# conda install html5lib
# conda install BeautifulSoup4
# Then restart Jupyter Notebook. (or use pip install if you aren't using the Anaconda Distribution)
import numpy as np
import pandas as pd
import os.path
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import *
from pandas.tseries.holiday import USFederalHolidayCalendar
from pylab import text
from mpl_toolkits.axes_grid.anchored_artists import AnchoredText

#You'll need to first set your API token here and uncomment or set externally in your OS preferably.
#%env TIINGOTOKEN inserttiingotokenhere

# Some formatting
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_seq_items', 2)

# Only pull fresh PDUFA data
one_hour_ago = datetime.now() - timedelta(hours=1)
if os.path.exists("history.csv"):
    filetime = datetime.fromtimestamp(os.path.getctime("history.csv"))
    if filetime < one_hour_ago:
        histdata = pd.read_html("https://www.biopharmcatalyst.com/calendars/historical-catalyst-calendar")
        histdata[0].to_csv('history.csv',index=False)
else:
    histdata = pd.read_html("https://www.biopharmcatalyst.com/calendars/historical-catalyst-calendar")
    histdata[0].to_csv('history.csv',index=False)

# Create master PDUFA dataframe
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
df = pd.read_csv('history.csv').set_index('Ticker')
df.index.name=None
df[["Date","Catalyst"]] = df.Catalyst.str.extract('(?P<Date>[0-9]{2}\/[0-9]{2}\/[0-9]{4})(?P<Catalyst>.*)', expand=True)
df['Date'] = pd.to_datetime(df['Date'])
df['day_of_week'] = df['Date'].dt.day_name()
df["Original_PDUFA"] = df["Date"]
df["Date"] = df["Date"].map(lambda x : x + 0*us_bd)
df["Past"] = df["Date"] - DateOffset(weeks=3)
df["Future"] = df["Date"] + DateOffset(weeks=1)    

# Create local PDUFA dataframe for comparison
ticker="BHC"
pdufadf = df.loc[ticker]
length = len(pdufadf.index)
count = 0
tiingotoken = os.environ['TIINGOTOKEN']
oldestpdufa = df.loc[ticker]["Date"].min().date() - timedelta(days=30)
latestpdufa = df.loc[ticker]["Date"].max().date() + timedelta(days=15)
stockdf = pd.DataFrame(pd.read_json(f"https://api.tiingo.com/tiingo/daily/{ticker}/prices?startDate={oldestpdufa}&endDate={latestpdufa}&token={tiingotoken}"))

fig, axes = plt.subplots(nrows=length, ncols=1,figsize=(14,length * 2))
#set hspace and wspace to 0 for stacked "sparklines" of sorts
fig.subplots_adjust(hspace=0, wspace=0)

# Create plot data
allplots=[]
e=[]
while (count < length):
    #Initialize variable for nth PDUFA
    stage = pdufadf.iloc[count]["Stage"]
    catalyst = pdufadf.iloc[count]["Catalyst"]
    drug = pdufadf.iloc[count]["Drug"]
    
    #Annotation
    tooltip = f"stage:{stage} \n{drug}"
    at = AnchoredText(tooltip,
                      prop=dict(size=10), frameon=True,
                      loc=2, 
                      )
    at.patch.set_boxstyle("round,pad=0.2,rounding_size=0.2")
    axes[count].add_artist(at)
    axes[count].margins(0.0, 0.0)
    axes[count].minorticks_on()
    axes[count].grid(which="major", axis="x")
    axes[count].grid(which="minor", axis="x")

    pdufa=pdufadf.iloc[count]["Date"]
    indexvalue = stockdf.index[stockdf["date"] == pdufadf.iloc[count]["Date"]][0]
    mydata = stockdf.iloc[indexvalue - 20:indexvalue + 5,].copy()
    allplots.append(mydata)
    allplots[count]["pdufa"] = allplots[count]["date"] == pdufa
    allplots[count].reset_index(inplace=True)
    axes[count].set_ylim(allplots[count]["close"].min() * .99 , (allplots[count]["close"].max()-allplots[count]["close"].min()) * 0.5 + allplots[count]["close"].max())
    axes[count].hlines(allplots[count]["close"].max() * 1.01, allplots[count].head(1).index, allplots[count].tail(1).index, linestyle="-", lw=1, color='black')
    if count % 2 == 0:
        axes[count].set_facecolor((0.91, 0.91, 0.91))  
    axes[count].vlines(allplots[count][allplots[count]["pdufa"] == True].index, allplots[count]["close"].min() * .99, allplots[count]["close"].max() * 1.01, linestyle="--", color='black')
    axes[count].plot(allplots[count].index,allplots[count]["close"],c=np.random.uniform(low=.25, high=.7, size=(3,)), lw=2, label=pdufa) 
    plt.minorticks_on()
#     axes[count].axes.get_xaxis().set_visible(False) # remove x axis
    i = 0
    total = len(allplots[count].index) - 1
    d=[]
    ecolumns=[]
    while (i < total):
        n = i + 1
        while (n < total):  
            rangevalue=((allplots[count].iloc[n]["close"] - allplots[count].iloc[i]["close"])/allplots[count].iloc[i]["close"] * 100)
            d.append(rangevalue)
            
            rangetest = f"{i} - {n}"
            ecolumns.append(rangetest)
            n=n + 1
        i = i + 1

    e.append(d)
    count = count + 1
#     calcdf = pd.DataFrame(e).transpose()
pdufaint=[]
for x in allplots[:]:
    pdufaint.append((x[x["pdufa"] == True].index.values[0]))
pdufaint=min(pdufaint)
testdf=pd.DataFrame(e)
testdf.columns=ecolumns
calcdf=testdf.transpose()
calcdf[["StartRange","EndRange"]] = calcdf.index.to_series().str.split(pat = " - ",expand=True)
calcdf = calcdf.astype({"StartRange": int, "EndRange": int})
calcdf["TotalRange"] = calcdf["EndRange"] - calcdf["StartRange"]
calcdf['Total_POS'] = (calcdf.iloc[:, : len(pdufadf.index)] > 0).sum(axis=1)
calcdf['Total_NEG'] = (calcdf.iloc[:, : len(pdufadf.index)] <= 0).sum(axis=1)
calcdf['Mean'] = calcdf.mean(axis=1)
calcdf['Stddev'] = calcdf.std(axis=1)
calcdf['Variance'] = calcdf.var(axis=1)
calcdf['Winner']=calcdf['Mean']/calcdf['Stddev']*calcdf["Total_POS"]/calcdf["Total_NEG"]
# calcdf[(calcdf['Total_POS'] >= (length - 1)) | (calcdf['Total_NEG'] >= (length - 1))]
avx=0
while (avx < length):
    axes[avx].axvspan(calcdf[(calcdf['Total_POS'] >= calcdf["Total_POS"].max() - 3)].sort_values(by=['Winner'],ascending=False).iloc[0]["StartRange"]
, calcdf[(calcdf['Total_POS'] >= calcdf["Total_POS"].max() - 3)].sort_values(by=['Winner'],ascending=False).iloc[0]["EndRange"]
, color='green', alpha=0.2)
    avx = avx + 1
calcdf[(calcdf['Total_POS'] >= calcdf["Total_POS"].max() - 3)&(calcdf["EndRange"] < pdufaint)].sort_values(by=['Winner'],ascending=False)

#CALCULATE BEST FIT DERIVATIVE PRE-PDUFA TO DETERMINE CONSISTENCY OF CURVE AND COMPARE TO TODAY.
#Figure out calculation for "winner". Is it stddev * mean? What about ranges to short?
# plt.show()


0          1          2          3          4          5  \
4 - 5    3.019146   0.754060  -3.219034   2.985075   1.819322   0.466472    
15 - 16  1.634877   1.761847  -2.060086   7.559614   1.254276   1.328168    
1 - 16   0.134228   1.086301  -21.092669  28.640777  12.476251  14.868256   
3 - 5    0.000000   0.462695  -4.356846   4.863222   2.916931   3.607937    
11 - 16  4.262753  -5.420666  -5.935697   3.163017   2.481246   4.808243    
6 - 7   -1.101322   0.524476  -3.510082   1.486564   3.126879   0.986651    
10 - 16  8.667152  -4.503991  -8.573718   7.614213   2.068966   4.389966    
14 - 16  4.701754  -0.534442  -5.389718   7.723577   1.659989   1.328168    
11 - 19  11.670161 -14.568041 -1.648805   5.450122   5.654934   3.148254    
9 - 19   15.965167 -13.542857 -3.712672   22.915485  6.268137   4.889406    
10 - 19  16.387473 -13.740023 -4.407051   10.000000  5.229885   2.736602    
6 - 19   17.327460 -11.829837 -10.903659  23.899371  10.102225  4.585026    
2 - 5   -5.023761   2.236610  -4.091540   5.182927   5.116580   6.161429    
11 - 17  5.101328  -7.284020  -5.935697   1.508516   4.270052   3.091013    
1 - 19   7.248322  -8.690404  -17.496542  31.492718  15.959468  13.048934   
10 - 17  9.541151  -6.385405  -8.573718   5.888325   3.850575   2.679590    
2 - 19   8.486083  -10.947616 -17.267684  32.134146  18.588083  11.028959   
3 - 19   14.224446 -12.492770 -17.496542  31.732523  16.106531  8.358388    
9 - 18   8.635704  -11.714286 -5.649718   18.491208  4.875218   3.376019    
4 - 19   17.673049 -12.238979 -16.515045  29.373134  14.868256  5.072886    
0 - 19   5.757776  -9.183673  -18.119423  30.778515  15.959468  14.267597   
6 - 18   9.911894  -9.965035  -12.696042  19.439680  8.659050   3.076030    
2 - 16   1.289885  -1.412596  -20.873786  29.268293  15.025907  12.815773   
5 - 19   14.224446 -12.895797 -13.738250  25.623188  12.815773  4.585026    
3 - 16   6.647605  -3.123193  -21.092669  28.875380  12.618897  10.102225   
10 - 18  9.031318  -11.915621 -6.330128   6.040609   3.850575   1.254276    
3 - 17   7.505361  -5.031810  -21.092669  26.808511  14.584654  8.298256    
2 - 17   2.104549  -3.354915  -20.873786  27.195122  17.033679  10.967344   
1 - 17   0.939597  -0.905250  -21.092669  26.577670  14.439519  12.986198   
3 - 18   7.005004  -10.641990 -19.156293  26.990881  14.584654  6.794949    
...           ...         ...        ...        ...        ...       ...    
10 - 11  4.224326   0.969213  -2.804487   4.314721  -0.402299  -0.399088    
2 - 4   -7.807196   1.471454  -0.901526   2.134146   3.238342   5.668515    
10 - 14  3.787327  -3.990878  -3.365385  -0.101523   0.402299   3.021665    
0 - 8   -11.780278  5.642257  -12.079616  1.689801   9.119696   9.892200    
12 - 15  3.234880  -7.371975  -3.319502  -9.876543   2.095460   3.021665    
14 - 19  12.140351 -10.154394 -1.077944   10.111789  4.808243  -0.276702    
2 - 14  -3.258656  -0.882872  -16.366158  20.000000  13.147668  11.337030   
0 - 4   -10.125745  3.481393  -1.921757   1.086301   0.949968   8.750793    
0 - 17  -0.463269  -1.440576  -21.688401  25.890163  14.439519  14.204185   
15 - 17  2.452316  -0.243013  -2.060086   5.834602   3.021665  -0.332042    
11 - 14 -0.419287  -4.912479  -0.577082  -4.233577   0.807848   3.434459    
14 - 17  5.543860  -2.494062  -5.389718   5.995935   3.434459  -0.332042    
3 - 7   -3.716941  -0.231348  -10.650069  7.902736   8.750793   4.630186    
0 - 15  -2.845797  -1.200480  -20.041181  18.949909  11.082964  14.584654   
3 - 6   -2.644746  -0.751880  -7.399723   6.322188   5.453393   3.607937    
9 - 10  -0.362845   0.228571   0.726392   11.741350  0.986651   2.095460    
1 - 4   -8.859060   4.043452  -1.175657   1.638350   0.949968   7.590966    
14 - 18  5.052632  -8.254157  -3.067993   6.148374   3.434459  -1.715551    
3 - 10  -1.858470   1.445922  -13.692946  19.756839  10.336081  5.472038    
5 - 7   -3.716941  -0.690846  -6.579899   2.898551   5.668515   0.986651    
14 - 15  3.01

In [266]:
import numpy
a = np.array([[1,3,4,5], [3,4]])
np.concatenate(a)

array([1, 3, 4, 5, 3, 4])